In [1]:
!pip install torchinfo
!add-apt-repository ppa:jonathonf/ffmpeg-4 -y
!apt-get update
!apt-get -y install ffmpeg
!pip install ffmpeg-python pillow
!pip install einops
!pip install opencv-python
!apt-get -y install libgl1-mesa-glx
!apt-get -y install libglib2.0-0


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
/usr/bin/sh: 1: add-apt-repository: not found
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease            
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                      
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upg

In [2]:
from shot_detecion_selector import ShotDetection
from io_setup import setup_video_path

In [3]:
model = ShotDetection('autoshot')

Loading pretrained model from ./AutoShot/model_weight/ckpt_0_200_0.pth


/workspace/Shot_Detection/AutoShot/model.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load(pretrained_path, map_location=self.device)


Current model has 90 params, Updating 90 params


In [4]:
videos = setup_video_path("./my_vids")

In [5]:
videos

{'.ipynb_checkpoints': {}, 'cvpr08demo': './my_vids/cvpr08demo.mp4'}

In [6]:
prediction_scenes = model.run_model(video_path_dict=videos)

63it [00:04, 14.02it/s]


In [7]:
prediction_scenes

{'.ipynb_checkpoints': {},
 'cvpr08demo': array([[   0,  149],
        [ 150,  157],
        [ 158,  511],
        [ 512,  558],
        [ 559,  737],
        [ 738, 1324],
        [1325, 1389],
        [1390, 1513],
        [1514, 1786],
        [1787, 1827],
        [1828, 1897],
        [1898, 2062],
        [2063, 2349],
        [2350, 2368],
        [2370, 2578],
        [2580, 2824],
        [2825, 2981],
        [2982, 3034],
        [3035, 3093],
        [3094, 3102]], dtype=int32)}

In [8]:
len(prediction_scenes["cvpr08demo"])

20

In [9]:
import os
import ffmpeg
def cut_video(video_path, frame_ranges):
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    output_dir = os.path.join(os.path.dirname(video_path), base_name + "Autoshot")
    os.makedirs(output_dir, exist_ok=True)
    probe = ffmpeg.probe(video_path)
    video_streams = [stream for stream in probe['streams'] if stream['codec_type'] == 'video']
    frame_rate = eval(video_streams[0]['r_frame_rate'])
    
    for i, (start_frame, end_frame) in enumerate(frame_ranges):
        start_time = start_frame / frame_rate
        end_time = (end_frame + 1) / frame_rate
        output_file = os.path.join(output_dir, f"{base_name}_part_{i+1}.mp4")
        
        ffmpeg.input(video_path, ss=start_time, to=end_time).output(output_file).run()

    print(f"All video parts have been saved in the folder: {output_dir}")

In [10]:
cut_video("./my_vids/cvpr08demo.mp4",prediction_scenes["cvpr08demo"].tolist())

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

All video parts have been saved in the folder: ./my_vids/cvpr08demoAutoshot


frame=   59 fps=0.0 q=-1.0 Lsize=     135kB time=00:00:02.34 bitrate= 471.7kbits/s speed=11.3x    
video:112kB audio:20kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 2.392688%
[libx264 @ 0x559deb346b80] frame I:1     Avg QP:16.50  size: 16894
[libx264 @ 0x559deb346b80] frame P:22    Avg QP:18.34  size:  3578
[libx264 @ 0x559deb346b80] frame B:36    Avg QP:24.34  size:   501
[libx264 @ 0x559deb346b80] consecutive B-frames:  5.1% 37.3% 10.2% 47.5%
[libx264 @ 0x559deb346b80] mb I  I16..4: 40.6% 57.8%  1.7%
[libx264 @ 0x559deb346b80] mb P  I16..4:  0.5%  1.2%  0.1%  P16..4: 24.6% 11.5%  8.6%  0.0%  0.0%    skip:53.5%
[libx264 @ 0x559deb346b80] mb B  I16..4:  0.0%  0.1%  0.0%  B16..8: 15.1%  1.2%  0.2%  direct: 0.4%  skip:83.0%  L0:35.0% L1:54.9% BI:10.0%
[libx264 @ 0x559deb346b80] 8x8 transform intra:61.0% inter:75.9%
[libx264 @ 0x559deb346b80] coded y,uvDC,uvAC intra: 55.0% 62.0% 21.2% inter: 7.4% 14.0% 0.1%
[libx264 @ 0x559deb346b80] i16 v,h,dc,p: 66% 22% 10%  2%
[